# Import Required Libraries
Import the necessary libraries, including OpenAI and Claude API clients.

In [ ]:
# Import Required Libraries
import openai
import requests
from dotenv import load_dotenv
import os

# Load API keys from .env file
load_dotenv()
openai.api_key = os.getenv('OPENAI_API_KEY')
claude_api_key = os.getenv('CLAUDE_API_KEY')
gemini_api_key = os.getenv('GEMINI_API_KEY')

# Initialize LLMs
Initialize the Google, OpenAI and Claude language models using their respective API keys.

In [ ]:
# Initialize Gemini, ChatGPT, and Sonnet

# Test Gemini API call
def call_gemini_api(prompt):
    response = requests.post(
        'https://api.gemini.ai/v1/completions',
        headers={'Authorization': f'Bearer {gemini_api_key}'},
        json={'prompt': prompt, 'max_tokens': 150}
    )
    return response.json()

# Test ChatGPT API call
def call_chatgpt_api(prompt):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=150
    )
    return response.choices[0].text.strip()

# Test Sonnet API call
def call_sonnet_api(prompt):
    response = requests.post(
        'https://api.sonnet.ai/v1/completions',
        headers={'Authorization': f'Bearer {sonnet_api_key}'},
        json={'prompt': prompt, 'max_tokens': 150}
    )
    return response.json()

# Test the API calls
gemini_response = call_gemini_api("Hello, Gemini!")
chatgpt_response = call_chatgpt_api("Hello, ChatGPT!")
sonnet_response = call_sonnet_api("Hello, Sonnet!")

print(gemini_response, chatgpt_response, sonnet_response)

# Define Interaction Variable
Define a variable to limit the number of interactions between the two models.

In [ ]:
# Define Interaction Variable
interaction_limit = 10

# User-defined Variables
thing_to_create = "language game"  # Change this to "machine learning algorithm", "marketing campaign strategy", etc.
evaluation_criteria = "fun"  # Change this to "usefulness", "effectiveness", "uniqueness", etc.

# Initialize Feedback Variables
feedback_buffer = []
collective_feedback = []

# Create Critical Feedback and Creation/Evaluation Loop
Define a function where models take turns creating and critically analyzing each others creation, and a third model provides meta evaluation to improve the critical evaluation that is stored in a buffer between creation and evaluation rounds.

In [ ]:
# Function to Create Thing
def create_thing(model, feedback):
    prompt = f"Create a {thing_to_create} based on the following feedback: {feedback}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=150)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 150})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Thing
def evaluate_thing(model, thing):
    prompt = f"Evaluate the following {thing_to_create} for {evaluation_criteria}: {thing}"
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=50)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
        return response.json()['choices'][0]['text'].strip()

# Function to Evaluate Feedback using Google Gemini
def evaluate_feedback(feedback):
    prompt = f"Evaluate the usefulness of the following feedback for creating a good {thing_to_create}: {feedback}"
    response = requests.post('https://api.gemini.ai/v1/completions', headers={'Authorization': f'Bearer {gemini_api_key}'}, json={'prompt': prompt, 'max_tokens': 50})
    return response.json()['choices'][0]['text'].strip()

# Iterate Game Creation and Evaluation
Implement a loop to alternate between game creation and evaluation and meta evaluation and pruning of the feedback buffer based on the interaction variables

In [ ]:
# Main Interaction Loop
for i in range(interaction_limit):
    # Create Thing
    if i % 2 == 0:
        thing = create_thing('openai', collective_feedback)
    else:
        thing = create_thing('claude', collective_feedback)
    
    # Evaluate Thing
    if i % 2 == 0:
        feedback = evaluate_thing('claude', thing)
    else:
        feedback = evaluate_thing('openai', thing)
    
    # Add Feedback to Buffer
    feedback_buffer.append(feedback)
    
    # Evaluate Feedback Buffer
    useful_feedback = []
    for fb in feedback_buffer:
        evaluation = evaluate_feedback(fb)
        if "useful" in evaluation:
            useful_feedback.append(fb)
    
    # Update Collective Feedback
    collective_feedback = useful_feedback

    # Print Current Thing and Feedback
    print(f"Round {i+1}")
    print(f"{thing_to_create.capitalize()}: {thing}")
    print(f"Feedback: {feedback}")
    print(f"Collective Feedback: {collective_feedback}")
    print("\n")

    # Print Final Thing:
    if len(collective_feedback) > 0:
        final_thing = create_thing('openai', collective_feedback)
        print(f"Final {thing_to_create.capitalize()}: {final_thing}")
    else:
        print("No useful feedback received. Unable to create final thing.")

In [ ]:
# Function to Evaluate Game and Feedback
def evaluate_game_and_feedback(model, thing, feedback):
    prompt = f"Evaluate the following {thing_to_create} and its feedback for {evaluation_criteria}: {thing}\nFeedback: {feedback}\nProvide a new prompt and environment variable to improve the process."
    if model == 'openai':
        response = openai.Completion.create(engine="davinci", prompt=prompt, max_tokens=100)
        return response.choices[0].text.strip()
    elif model == 'claude':
        response = requests.post('https://api.claude.ai/v1/completions', headers={'Authorization': f'Bearer {claude_api_key}'}, json={'prompt': prompt, 'max_tokens': 100})
        return response.json()['choices'][0]['text'].strip()

# Initialize Improvement Buffer
improvement_buffer = []

# Main Interaction Loop with Improvement
for i in range(interaction_limit):
    # Create Thing
    if i % 2 == 0:
        thing = create_thing('openai', collective_feedback)
    else:
        thing = create_thing('claude', collective_feedback)
    
    # Evaluate Thing
    if i % 2 == 0:
        feedback = evaluate_thing('claude', thing)
    else:
        feedback = evaluate_thing('openai', thing)
    
    # Add Feedback to Buffer
    feedback_buffer.append(feedback)
    
    # Evaluate Feedback Buffer
    useful_feedback = []
    for fb in feedback_buffer:
        evaluation = evaluate_feedback(fb)
        if "useful" in evaluation:
            useful_feedback.append(fb)
    
    # Update Collective Feedback
    collective_feedback = useful_feedback

    # Evaluate Game and Feedback
    if i % 2 == 0:
        improvement = evaluate_game_and_feedback('openai', thing, feedback)
    else:
        improvement = evaluate_game_and_feedback('claude', thing, feedback)
    
    # Add Improvement to Buffer
    improvement_buffer.append(improvement)

    # Print Current Thing, Feedback, and Improvement
    print(f"Round {i+1}")
    print(f"{thing_to_create.capitalize()}: {thing}")
    print(f"Feedback: {feedback}")
    print(f"Collective Feedback: {collective_feedback}")
    print(f"Improvement: {improvement}")
    print("\n")

# Print Final Thing and Improvements
if len(collective_feedback) > 0:
    final_thing = create_thing('openai', collective_feedback)
    print(f"Final {thing_to_create.capitalize()}: {final_thing}")
else:
    print("No useful feedback received. Unable to create final thing.")

print("Improvements Buffer:")
for imp in improvement_buffer:
    print(imp)